In [1]:
#Define Dependencies need to perfrom ETL
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
#Extract the Olympic data on how athlete's performed
csv_file = "athlete_events.csv"
athlete_events_df = pd.read_csv(csv_file)
athlete_events_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
# Check the column headers 
athlete_events_df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [4]:
# Extract related fields for analysis from olympic data
athlete_events_df = athlete_events_df[["Sex", "NOC", "Sport", "Medal"]].copy()
athlete_events_df.head()
# 27116 Rows

,Sex,NOC,Sport,Medal
0,M,CHN,Basketball,NaN
1,M,CHN,Judo,NaN
2,M,DEN,Football,NaN
3,M,DEN,Tug-Of-War,Gold
4,F,NED,Speed Skating,NaN


In [5]:
#Extract regions key from NOC CSV

csv_file = "noc_regions.csv"
regions_df = pd.read_csv(csv_file)
regions_df = regions_df[["NOC", "region"]].copy()
regions_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [6]:
# Merge the two DataFrames together for Olypmics Data 
# and a list of regions dfined by NOC

athlete_events_noc_df = pd.merge(athlete_events_df, regions_df, on="NOC")

athlete_events_noc_df.head(10)



,Sex,NOC,Sport,Medal,region
0,M,CHN,Basketball,NaN,China
1,M,CHN,Judo,NaN,China
2,M,CHN,Boxing,NaN,China
3,M,CHN,Wrestling,NaN,China
4,F,CHN,Swimming,NaN,China
5,F,CHN,Swimming,NaN,China
6,M,CHN,Speed Skating,NaN,China
7,M,CHN,Speed Skating,NaN,China
8,M,CHN,Short Track Speed Skating,Silver,China
9,M,CHN,Short Track Speed Skating,NaN,China


In [7]:
# Rename Regions to Country to match GDP data for later merging on
athlete_events_noc_df = athlete_events_noc_df.rename(columns={"region":"Country"})
athlete_events_noc_df.head(10)


,Sex,NOC,Sport,Medal,Country
0,M,CHN,Basketball,NaN,China
1,M,CHN,Judo,NaN,China
2,M,CHN,Boxing,NaN,China
3,M,CHN,Wrestling,NaN,China
4,F,CHN,Swimming,NaN,China
5,F,CHN,Swimming,NaN,China
6,M,CHN,Speed Skating,NaN,China
7,M,CHN,Speed Skating,NaN,China
8,M,CHN,Short Track Speed Skating,Silver,China
9,M,CHN,Short Track Speed Skating,NaN,China


In [8]:
athlete_events_noc_df.count()

Sex        270767
NOC        270767
Sport      270767
Medal       39774
Country    270746
dtype: int64

In [9]:
# Extract the GDP Data from csv

csv_file = "countries of the world.csv"
countries_df = pd.read_csv(csv_file)
countries_df.head(227)

,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN
5,Angola,SUB-SAHARAN AFRICA,12127071,1246700,"9,7","0,13",0,"191,19",1900.0,"42,0","7,8","2,41","0,24","97,35",NaN,"45,11","24,2","0,096","0,658","0,246"
6,Anguilla,LATIN AMER. & CARIB,13477,102,"132,1","59,80","10,76","21,03",8600.0,"95,0","460,0",0,0,100,2,"14,17","5,34","0,04","0,18","0,78"
7,Antigua & Barbuda,LATIN AMER. & CARIB,69108,443,"156,0","34,54","-6,15","19,46",11000.0,"89,0","549,9","18,18","4,55","77,27",2,"16,93","5,37","0,038","0,22","0,743"
8,Argentina,LATIN AMER. & CARIB,39921833,2766890,"14,4","0,18","0,61","15,18",11200.0,"97,1","220,4","12,31","0,48","87,21",3,"16,73","7,55","0,095","0,358","0,547"
9,Armenia,C.W. OF IND. STATES,2976372,29800,"99,9","0,00","-6,47","23,28",3500.0,"98,6","195,7","17,55","2,3","80,15",4,"12,07","8,23","0,239","0,343","0,418"


In [10]:
# Apply some styling to the data frame

countries_df = countries_df[["Country", "Population", "GDP ($ per capita)"]].copy()
countries_df["GDP ($ per capita)"] = countries_df["GDP ($ per capita)"].map("${:,.2f}".format)
countries_df["Population"] = countries_df["Population"].map("{:,.2f}".format)
countries_df.head(10)

,Country,Population,GDP ($ per capita)
0,Afghanistan,"31,056,997.00",$700.00
1,Albania,"3,581,655.00","$4,500.00"
2,Algeria,"32,930,091.00","$6,000.00"
3,American Samoa,"57,794.00","$8,000.00"
4,Andorra,"71,201.00","$19,000.00"
5,Angola,"12,127,071.00","$1,900.00"
6,Anguilla,"13,477.00","$8,600.00"
7,Antigua & Barbuda,"69,108.00","$11,000.00"
8,Argentina,"39,921,833.00","$11,200.00"
9,Armenia,"2,976,372.00","$3,500.00"


In [11]:
# Clean the data of the Olypmic data to focus on countries that won medals
# Igore the ones that no none as it will not coorelate to GDP
# Drop the NAN values from medals/country data frame


country_medals_df = athlete_events_noc_df
country_medals_df = country_medals_df.dropna()
country_medals_df.head()

,Sex,NOC,Sport,Medal,Country
8,M,CHN,Short Track Speed Skating,Silver,China
10,M,CHN,Short Track Speed Skating,Bronze,China
11,M,CHN,Short Track Speed Skating,Bronze,China
12,F,CHN,Softball,Silver,China
17,F,CHN,Basketball,Bronze,China


In [12]:
list(country_medals_df)

['Sex', 'NOC', 'Sport', 'Medal', 'Country']

In [13]:
# Group by both country and medal
 
g_country_medal_df = country_medals_df.groupby(["Country", "Medal"])

g_country_medal_df.head()

,Sex,NOC,Sport,Medal,Country
8,M,CHN,Short Track Speed Skating,Silver,China
10,M,CHN,Short Track Speed Skating,Bronze,China
11,M,CHN,Short Track Speed Skating,Bronze,China
12,F,CHN,Softball,Silver,China
17,F,CHN,Basketball,Bronze,China
33,F,CHN,Fencing,Silver,China
41,F,CHN,Gymnastics,Silver,China
53,F,CHN,Swimming,Bronze,China
63,F,CHN,Rhythmic Gymnastics,Silver,China
68,M,CHN,Shooting,Gold,China


In [14]:
# Perform a count of medal's / country by grouping data
# g in g_country_medal_df means "group"
g_country_medal_df.count()

Sex   NOC  Sport
Country              Medal                    
Afghanistan          Bronze     2     2      2
Algeria              Bronze     8     8      8
                     Gold       5     5      5
                     Silver     4     4      4
Argentina            Bronze    91    91     91
                     Gold      91    91     91
                     Silver    92    92     92
Armenia              Bronze     9     9      9
                     Gold       2     2      2
                     Silver     5     5      5
Australia            Bronze   522   522    522
                     Gold     368   368    368
                     Silver   459   459    459
Austria              Bronze   156   156    156
                     Gold     108   108    108
                     Silver   186   186    186
Azerbaijan           Bronze    25    25     25
                     Gold       7     7      7
                     Silver    12    12     12
Bahamas              Bronze    15    15     15
                     Gold      14    14     14
                     Silver    11    11     11
Bahrain              Bronze     1     1      1
                     Gold       1     1      1
                     Silver     1     1      1
Barbados             Bronze     1     1      1
Belarus              Bronze    71    71     71
                     Gold      24    24     24
                     Silver    44    44     44
Belgium              Bronze   173   173    173
...                           ...   ...    ...
UK                   Gold     678   678    678
                     Silver   739   739    739
USA                  Bronze  1358  1358   1358
                     Gold    2638  2638   2638
                     Silver  1641  1641   1641
Uganda               Bronze     2     2      2
                     Gold       2     2      2
                     Silver     3     3      3
Ukraine              Bronze   100   100    100
                     Gold      47    47     47
                     Silver    52    52     52
United Arab Emirates Bronze     1     1      1
                     Gold       1     1      1
Uruguay              Bronze    30    30     30
                     Gold      31    31     31
                     Silver     2     2      2
Uzbekistan           Bronze    17    17     17
                     Gold      10    10     10
                     Silver     7     7      7
Venezuela            Bronze    10    10     10
                     Gold       2     2      2
                     Silver     3     3      3
Vietnam              Gold       1     1      1
                     Silver     3     3      3
Virgin Islands, US   Silver     1     1      1
Zambia               Bronze     1     1      1
                     Silver     1     1      1
Zimbabwe             Bronze     1     1      1
                     Gold      17    17     17
                     Silver     4     4      4

[327 rows x 3 columns]

In [15]:
# Bronze, Silver, Gold into  data frame by count of medals
bsg_grouped = g_country_medal_df.count()

In [16]:
# View the Bronze, Silver, Gold data frame
bsg_grouped.head()

Sex  NOC  Sport
Country     Medal                  
Afghanistan Bronze    2    2      2
Algeria     Bronze    8    8      8
            Gold      5    5      5
            Silver    4    4      4
Argentina   Bronze   91   91     91

In [17]:
# Joining the two data frames between Bronze,Silver, Gold Grouped and GDP Data
join_gdp = pd.merge(bsg_grouped, countries_df, on="Country", how='outer')

In [18]:
join_gdp = join_gdp.sort_values(["Country","GDP ($ per capita)"], ascending=True)
join_gdp.head()

,Country,Sex,NOC,Sport,Population,GDP ($ per capita)
0,Afghanistan,2.0,2.0,2.0,NaN,NaN
327,Afghanistan,NaN,NaN,NaN,"31,056,997.00",$700.00
328,Albania,NaN,NaN,NaN,"3,581,655.00","$4,500.00"
1,Algeria,8.0,8.0,8.0,NaN,NaN
2,Algeria,5.0,5.0,5.0,NaN,NaN


In [19]:
join_gdp = join_gdp.rename(columns={"Sport":"Total_Medals"})


In [20]:
join_gdp = join_gdp.drop(columns=['Sex', 'NOC'])
join_gdp.head(10)

,Country,Total_Medals,Population,GDP ($ per capita)
0,Afghanistan,2.0,NaN,NaN
327,Afghanistan,NaN,"31,056,997.00",$700.00
328,Albania,NaN,"3,581,655.00","$4,500.00"
1,Algeria,8.0,NaN,NaN
2,Algeria,5.0,NaN,NaN
3,Algeria,4.0,NaN,NaN
329,Algeria,NaN,"32,930,091.00","$6,000.00"
330,American Samoa,NaN,"57,794.00","$8,000.00"
331,Andorra,NaN,"71,201.00","$19,000.00"
332,Angola,NaN,"12,127,071.00","$1,900.00"


In [21]:
# Trying to join the rows on country and keeping the values
# This is NOT WORKING 
join_gdp.groupby('Country')[['Total_Medals', 'Population', 'GDP ($ per capita)']].first().reset_index()

,Country,Total_Medals,Population,GDP ($ per capita)
0,Afghanistan,2.0,NaN,NaN
1,Afghanistan,NaN,"31,056,997.00",$700.00
2,Albania,NaN,"3,581,655.00","$4,500.00"
3,Algeria,8.0,NaN,NaN
4,Algeria,NaN,"32,930,091.00","$6,000.00"
5,American Samoa,NaN,"57,794.00","$8,000.00"
6,Andorra,NaN,"71,201.00","$19,000.00"
7,Angola,NaN,"12,127,071.00","$1,900.00"
8,Anguilla,NaN,"13,477.00","$8,600.00"
9,Antigua & Barbuda,NaN,"69,108.00","$11,000.00"
